<a href="https://colab.research.google.com/github/Spoor2709/Nasa-From-Curious-Minds-Come-Helping-Hands/blob/DataScience/DataScience/final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [0]:

%matplotlib inline
import tensorflow.keras
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
import numpy as np
import os

In [0]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
#!pip install tensorflow-gpu==1.14.0

In [73]:
#import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import layers
tf.__version__

'1.15.0-rc3'

In [0]:
def path_join(dirname, filenames):
    return [os.path.join(dirname, filename) for filename in filenames]

In [0]:
def load_images(image_paths):
    # Load the images from disk.
    images = [plt.imread(path) for path in image_paths]

    # Convert to a numpy array and return it.
    return np.asarray(images)

In [0]:
train_dir = '../gdrive/My Drive/NASA/train'
test_dir = '../gdrive/My Drive/NASA/test'

In [0]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg19 import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [0]:
model = VGG19(include_top=True, weights='imagenet')

In [79]:
input_shape = model.layers[0].output_shape[0][1:3]
input_shape

(224, 224)

In [0]:
###try one pic
def predict(image_path):
  
  # convert image to .jpg if it is .png
    if image_path.lower().endswith('.png'):
        img = PIL.Image.open(image_path)
        rgb_im = img.convert('RGB')
        image_path=image_path.replace("png", "jpg").replace("PNG","jpg")
        rgb_im.save(image_path, quality=95)

    # Load and resize the image using PIL.
    img = PIL.Image.open(image_path)
    print('input_shape')
    img_resized = img.resize(input_shape, PIL.Image.LANCZOS)

    # Plot the image.
    #plt.imshow(img_resized)
    #plt.show()

    # Convert the PIL image to a numpy-array with the proper shape.
    img_array = np.expand_dims(np.array(img_resized), axis=0)

    # Use the VGG16 model to make a prediction.
    # This outputs an array with 1000 numbers corresponding to
    # the classes of the ImageNet-dataset.
    pred = model.predict(img_array)
    
    # Decode the output of the VGG16 model.
    pred_decoded = decode_predictions(pred)[0]

    # Print the predictions.
    for code, name, score in pred_decoded:
        print("{0:>6.2%} : {1}".format(score, name))

In [112]:
predict(image_path= '../gdrive/My Drive/NASA/test/after/after_23.jpg')

input_shape
46.63% : throne
 7.95% : breastplate
 6.50% : cuirass
 1.94% : vault
 1.62% : palace


In [82]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [0]:
datagen_train = ImageDataGenerator(
      rescale=1./255,
      rotation_range=180,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=[0.9, 1.5],
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest')

In [0]:
datagen_test = ImageDataGenerator(rescale=1./255) 

In [0]:
batch_size = 10 

In [0]:
 if True:
    save_to_dir = None
else:
    save_to_dir='augmented_images/'

In [117]:
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    save_to_dir=save_to_dir)

Found 77 images belonging to 2 classes.


In [118]:
generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

Found 25 images belonging to 2 classes.


In [119]:
steps_test = generator_test.n / batch_size
steps_test

2.5

In [0]:
image_paths_train = path_join(train_dir, generator_train.filenames)
image_paths_test = path_join(test_dir, generator_test.filenames)

In [0]:
cls_train = generator_train.classes
cls_test = generator_test.classes

In [122]:
class_names = list(generator_train.class_indices.keys())
class_names

['after', 'before']

In [123]:
num_classes = generator_train.num_classes
num_classes

2

In [0]:
from sklearn.utils.class_weight import compute_class_weight

In [0]:
 class_weight = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(cls_train),
                                    y=cls_train)

In [126]:
class_weight 

array([0.9625    , 1.04054054])

In [127]:
model.summary() 

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [0]:
transfer_layer = model.get_layer('fc2') 

In [0]:
from tensorflow.keras import layers

In [130]:
conv_model = Model(inputs=model.input, outputs=transfer_layer.output)
model.input

<tf.Tensor 'input_3:0' shape=(?, 224, 224, 3) dtype=float32>

In [0]:
new_model = Sequential()
new_model.add(conv_model)

# Add the final layer for the actual classification.
new_model.add(Dense(num_classes, activation='softmax'))

In [132]:
new_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_5 (Model)              (None, 4096)              139570240 
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 8194      
Total params: 139,578,434
Trainable params: 139,578,434
Non-trainable params: 0
_________________________________________________________________


In [0]:
optimizer = Adam(lr=1e-5)
metrics = ['categorical_accuracy']
loss = 'categorical_crossentropy'
new_model.compile(optimizer = optimizer,loss=loss, metrics=metrics)
epochs = 10
steps_per_epoch = 100


In [0]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

In [136]:
history = new_model.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  class_weight=class_weight,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

Epoch 1/10
100/100 [==============================] - 2196s 22s/step - loss: 0.4212 - categorical_accuracy: 0.7846 - val_loss: 1.0793 - val_categorical_accuracy: 0.6800
Epoch 2/10
100/100 [==============================] - 2192s 22s/step - loss: 0.2419 - categorical_accuracy: 0.8859 - val_loss: 1.1584 - val_categorical_accuracy: 0.5600
Epoch 3/10
100/100 [==============================] - 2177s 22s/step - loss: 0.1285 - categorical_accuracy: 0.9481 - val_loss: 0.9108 - val_categorical_accuracy: 0.6800
Epoch 4/10
100/100 [==============================] - 2169s 22s/step - loss: 0.0357 - categorical_accuracy: 0.9875 - val_loss: 1.1099 - val_categorical_accuracy: 0.7200
Epoch 5/10
100/100 [==============================] - 2174s 22s/step - loss: 0.0322 - categorical_accuracy: 0.9917 - val_loss: 1.1059 - val_categorical_accuracy: 0.6000
Epoch 6/10
100/100 [==============================] - 2177s 22s/step - loss: 0.0048 - categorical_accuracy: 1.0000 - val_loss: 1.6649 - val_categorical_acc

In [137]:
from keras.models import load_model

model_json = model.to_json()
with open("/../gdrive/My Drive/NASA/nasa_challenge_full.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/gdrive/My Drive/NASA/nasa_challenge_full.h5")
print("Saved model to disk")
 

Using TensorFlow backend.


Saved model to disk


In [0]:
model = load_model('/gdrive/My Drive/NASA_challenge_full.h5')

In [0]:
'''from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.fit(x, y, validation_split=0.2, callbacks=[early_stopping])'''